In [4]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# os.getcwd()

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# t = PromptTemplate.from_template("What is the capital of {country}?")
# t = PromptTemplate(
#     template = "What is the capital of {country}?",
#     input_variables = ["country"]
# )

# t.format(country = "France")

In [8]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [10]:
# 4.1.
# example_template = """
#     Human: {question},
#     AI: {answer}
# """

example_prompt = PromptTemplate.from_template("Human: {question}\n AI: {answer}")
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")
prompt.format(country="Turkey")

# chat.predict("What do you know about France?")

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [17]:
# 4.2. 
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are geograpphy expert, you give short answers"),
    example_prompt,
    ("human", "What do you know about {country}")

])

# prompt.format(country="Germany")
# prompt.format(country="Turkey")

# chat.predict("What do you know about France?")

chain = final_prompt | chat

# chain.invoke({
#     "country": "Germany"
# })
chain.invoke({
    "country": "Thailand"
})

I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [20]:
# 4.3
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
# from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import example_selector
# from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
# from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        # return choice(examples)
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\n AI: {answer}")

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=80
# )
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")
prompt.format(country="Brazil")

# chain = prompt | chat

# chain.invoke({
#     "country": "Germany"
# })

'Human: What do you know about Italy?\n AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [22]:
import os
os.getcwd()

'd:\\Library\\Code\\NomadCoders_FullstackGPT'

In [27]:
# 4.3
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import load_prompt

from langchain.prompts.pipeline import PipelinePromptTemplate

# prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# prompt.format(country="Germany")

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
    ("final", final)
]

# full_prompt = PipelinePromptTemplate(
#     final_prompt=final,
#     pipleline_prompts=prompts,
# )
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrg! That is a secrete!!",
#     question="What is your favorate food?"
# )

chain = full_prompt | chat
# chain.invoke({
#     "character"="Pirate",
#     "example_question"="What is your location?",
#     "example_answer"="Arrrg! That is a secrete!!",
#     "question"="What is your favorate food?"
# })
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


In [ ]:
# 4.5
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")
chat.predict("How do you make italian pasta")

In [ ]:
# 4.6
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

In [28]:
# Homework
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "movie": "The Godfather",
        "answer": """
        Here is what I know:
        Director: Francis Ford Coppola
        Actors: Marlon Brando, Al Pacino
        Budget: $6 million
        Profit: $245 million
        Genre: Crime
        Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
        """,
    },
    {
        "movie": "Pulp Fiction",
        "answer": """
        I know this:
        Director: Quentin Tarantino
        Actors: John Travolta, Uma Thurman
        Budget: $8 million
        Profit: $213.9 million
        Genre: Crime
        Synopsis: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.
        """,
    },
    {
        "movie": "Fight Club",
        "answer": """
        I know this:
        Director: David Fincher
        Actors: Brad Pitt, Edward Norton
        Budget: $63 million
        Profit: $100.9 million
        Genre: Drama
        Synopsis: An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {movie}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are movie expert, you give short answers"),
    example_prompt,
    ("human", "What do you know about {movie}")

])

chain = final_prompt | chat

chain.invoke({
    "movie": "Dallas Buyers Club"
})


        I know this:
        Director: Jean-Marc Vallée
        Actors: Matthew McConaughey, Jared Leto
        Budget: $5 million
        Profit: $55.2 million
        Genre: Drama
        Synopsis: In 1985 Dallas, electrician and hustler Ron Woodroof works around the system to help AIDS patients get the medication they need after he is diagnosed with the disease.
        

AIMessageChunk(content='\n        I know this:\n        Director: Jean-Marc Vallée\n        Actors: Matthew McConaughey, Jared Leto\n        Budget: $5 million\n        Profit: $55.2 million\n        Genre: Drama\n        Synopsis: In 1985 Dallas, electrician and hustler Ron Woodroof works around the system to help AIDS patients get the medication they need after he is diagnosed with the disease.\n        ')